In [1]:
import pandas as pd

In [2]:
# reset all data
full_filepath = '/'
current_directory = ''
file_directory = pd.DataFrame()

def read_txt_to_list(filepath):
    """read txt files into a list.   Each item in the list is a line in the txt file"""
    with open(filepath) as input_file:
        data = input_file.readlines()
        data = [line.rstrip() for line in data]
    return data

def go_up_one_directory(full_filepath, current_directory):
    """ go up one directory"""
    position =  len(full_filepath)-len(current_directory)-1
    full_filepath = full_filepath[0:position]
    current_directory = full_filepath.split('/')[-1]
    return full_filepath, current_directory

def is_command(command):
    """ check if the line is a command"""
    if command.startswith('$ cd'):
        return True
    return False

def record_file_attributes(command, full_filepath, file_directory):
    if command[0].isdigit():
        file_size, file_name = command.split(' ')
        new_row = pd.DataFrame({'filepath': full_filepath, 'file_name':file_name, 'file_size':int(file_size)}, index =[0])
        file_directory = pd.concat([new_row, file_directory.loc[:]]).reset_index(drop=True)
    return file_directory

def read_command(command, full_filepath, current_directory):

    """ read the command.   Either returns directory to the top. or go up and down one directory """
    if command.endswith('cd /'):   #command to reset the diredirectory to the top
         full_filepath = ''
         current_directory = ''
         return full_filepath, current_directory
    if command.startswith('$ cd'):
        if command.endswith('..'):
            full_filepath, current_directory = go_up_one_directory(full_filepath, current_directory)
            return full_filepath, current_directory
        else:
            current_directory = command.split(' ')[-1]
            full_filepath += '/' + current_directory
            return full_filepath, current_directory

In [3]:
data = read_txt_to_list('Data/day7_input.txt')

#data = read_txt_to_list('Data/day7_test.txt')

In [4]:
# reset all data
full_filepath = '/'
current_directory = ''
file_directory = pd.DataFrame()

#run the commands line by line

for row in data:
    if is_command(row):
        #print(row)
        full_filepath, current_directory = read_command(row, full_filepath, current_directory)
    else:
        file_directory = record_file_attributes(row, full_filepath, file_directory)

#sum all the folders
directory_size = file_directory.sort_values('filepath')

#split the directories into a column dimension for each level of directory
split_directory = directory_size['filepath'].str.split('/',expand=True)
split_directory['size'] = directory_size['file_size']
clean_dir_size = split_directory.copy()


In [5]:
part1 = pd.DataFrame(columns=['folder', 'size'])

#Get a list of column names to aggregate the directories
directory_columns = clean_dir_size.columns[:-1].values

for column_name in directory_columns:
    summed_dir_size = clean_dir_size[[column_name,'size']].groupby(column_name).sum().reset_index()
    acceptable_folder =  summed_dir_size[summed_dir_size['size'] <= 100000]
    acceptable_folder.columns = ['folder', 'size']
    part1 = pd.concat([part1, acceptable_folder])
    #print(acceptable_folder)

part1['size'].sum()

950635